In [7]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
from sklearn.neighbors import BallTree
import glob
import os
from datetime import datetime, timedelta
from scipy.spatial import cKDTree


def add_public_school_dataset(file_path):
    """
    This function takes a TSV file input, joins it with a public school dataset,
    and adds 3 new features to the initial input.
    """
    # Load haunted places dataset from TSV
    haunted_df = pd.read_csv(file_path, sep="\t", on_bad_lines="skip")

    # Load the Public Schools GeoJSON File
    schools_gdf = gpd.read_file("../data/Public_Schools_-5088709809754466635.geojson")

    # Extract latitude and longitude from the geometry
    schools_gdf["latitude"] = schools_gdf.geometry.y
    schools_gdf["longitude"] = schools_gdf.geometry.x

    # Convert to DataFrame (removing geometry)
    schools_df = schools_gdf.drop(columns=["geometry"])

    
    # Convert Haunted Places to GeoDataFrame
    haunted_gdf = gpd.GeoDataFrame(
        haunted_df,
        geometry=gpd.points_from_xy(haunted_df["longitude"], haunted_df["latitude"]),
        crs="EPSG:4326"
    )

    # Convert Schools CSV to GeoDataFrame
    schools_gdf = gpd.GeoDataFrame(
        schools_df,
        geometry=gpd.points_from_xy(schools_df["longitude"], schools_df["latitude"]),
        crs="EPSG:4326"
    )

    # Convert both datasets to EPSG:3857 (meters-based projection)
    schools_gdf = schools_gdf.to_crs(epsg=3857)
    haunted_gdf = haunted_gdf.to_crs(epsg=3857)

    # Create a 10-mile buffer around each haunted place
    haunted_gdf["buffer_10_miles"] = haunted_gdf.geometry.buffer(10 * 1609.34)

    # Count schools within the buffer of each haunted place
    haunted_gdf["schools_within_10_miles"] = haunted_gdf["buffer_10_miles"].apply(
        lambda buffer: schools_gdf[schools_gdf.geometry.within(buffer)].shape[0]
    )

    # Compute nearest school distances using BallTree
    def compute_nearest_distances(haunted_gdf, schools_gdf):
        """
        Computes the nearest school distance for each haunted location using BallTree.
        """
        # Drop rows with NaN values in latitude/longitude
        haunted_gdf = haunted_gdf.dropna(subset=["latitude", "longitude"]).copy()
        schools_gdf = schools_gdf.dropna(subset=["latitude", "longitude"]).copy()

        # Convert degrees to radians for BallTree
        haunted_coords = np.radians(haunted_gdf[["latitude", "longitude"]].to_numpy())
        school_coords = np.radians(schools_gdf[["latitude", "longitude"]].to_numpy())

        if school_coords.shape[0] == 0:
            print("⚠ No valid school coordinates found. Setting distance to NaN.")
            haunted_gdf["distance_to_nearest_school_km"] = np.nan
            return haunted_gdf

        # Build BallTree with school locations
        tree = BallTree(school_coords, metric="haversine")

        # Query the nearest school for each haunted location
        distances, _ = tree.query(haunted_coords, k=1)

        # Convert from radians to kilometers (Earth's radius = 6371 km)
        haunted_gdf.loc[:, "distance_to_nearest_school_km"] = distances[:, 0] * 6371

        return haunted_gdf

    # Compute distances using BallTree
    haunted_gdf = compute_nearest_distances(haunted_gdf, schools_gdf)

    # Add feature 3: Check if haunted place is a school
    haunted_gdf.loc[:, "is_haunted_place_a_school?"] = haunted_gdf["location"].str.contains(
        "school", case=False, na=False
    ).map({True: "Yes", False: "No"})

    # Save the updated dataset
    haunted_gdf.to_csv("../data/haunted_places_with_new_features.csv", index=False)

    return haunted_gdf

# Example usage
first_dataset_added = add_public_school_dataset("../data/haunted_places.tsv")







In [35]:
def add_weather_dataset(file):
    """
    This function loads a CSV file containing haunted places, merges it with a weather dataset,
    and adds relevant weather features to each haunted place using the nearest weather station.

    - Extracts weather data from .dly files
    - Filters for US-based stations
    - Keeps only PRCP, SNWD, TMAX, and TMIN values
    - Converts wide format to long format
    - Filters for the last 3 years
    - Merges with station location data
    - Uses KDTree to efficiently find the nearest station for each haunted place
    - Adds derived weather features such as Diurnal Temperature Range (DTR)

    Returns:
        final_df (pd.DataFrame): Processed dataset with added weather features.
    """

    # Path to the folder containing .dly files
    folder_path = "../data/GLOBAL_HISTORICAL_CLIMATOLOGY_NETWORK"  

    # Get a list of all .dly files
    file_list = glob.glob(os.path.join(folder_path, "*.dly"))

    # Define fixed-width columns and corresponding field names
    column_specs = [
        (0, 11),  # ID (Station ID)
        (11, 15),  # YEAR
        (15, 17),  # MONTH
        (17, 21),  # ELEMENT
    ] + [(21 + i * 8, 26 + i * 8) for i in range(31)]  # 31 DAYS (Values)

    # Column names
    column_names = ["ID", "YEAR", "MONTH", "ELEMENT"] + [f"DAY{i}" for i in range(1, 32)]

    # Elements of interest
    desired_elements = {"PRCP", "SNWD", "TMAX", "TMIN"}  # Precip, Snow Depth, Max/Min Temp

    # Get today's date and compute cutoff for the last 2 years
    today = datetime.today()
    last_3_years = today - timedelta(days=2 * 365)  # Approximate 2 years

    def parse_dly_file(path_to_file):
        """
        Parses a single .dly weather data file, filters relevant elements,
        and converts it to a clean long-format DataFrame.
        """
        try:
            df = pd.read_fwf(path_to_file, colspecs=column_specs, header=None, names=column_names)

            # Filter only US stations (ID starts with 'US')
            df = df[df["ID"].str.startswith("US")]

            # Keep only the required weather elements
            df = df[df["ELEMENT"].isin(desired_elements)]

            # Convert wide format to long format
            df_long = df.melt(id_vars=["ID", "YEAR", "MONTH", "ELEMENT"], 
                              var_name="DAY", value_name="VALUE")

            # Extract numerical day values
            df_long["DAY"] = df_long["DAY"].str.extract(r"(\d+)").astype(int)

            # Remove invalid/missing values (-9999 represents missing data)
            df_long = df_long[df_long["VALUE"] != -9999]

            # Keep only valid day values (1-31)
            df_long = df_long[(df_long["DAY"] >= 1) & (df_long["DAY"] <= 31)]

            # Convert to proper date format and remove invalid dates
            df_long["DATE"] = pd.to_datetime(df_long[["YEAR", "MONTH", "DAY"]], errors="coerce")
            df_long.dropna(subset=["DATE"], inplace=True)

            # Filter for the last 3 years only
            df_long = df_long[df_long["DATE"] >= last_3_years]

            # Drop unnecessary columns
            df_long.drop(columns=["YEAR", "MONTH", "DAY"], inplace=True)

            return df_long
        except Exception as e:
            print(f"Error processing {path_to_file}: {e}")
            return pd.DataFrame()  # Return empty DataFrame in case of errors

    # Efficiently process files and store results in a list
    all_data = [parse_dly_file(file) for file in file_list]
    
    # Combine all processed data
    combined_data = pd.concat(all_data, ignore_index=True) 

    # Load station metadata (ID, Latitude, Longitude)
    stations = pd.read_csv("../data/ghcnd-stations.csv", usecols=[0, 1, 2], 
                           header=None, names=["ID", "latitude", "longitude"])

    # Merge weather data with station locations
    merged_df = combined_data.merge(stations, on="ID", how="left")
    
    # Pivot to get a structured format with weather elements as columns
    merged_df_pivot = merged_df.pivot_table(index="ID", columns="ELEMENT", values="VALUE", aggfunc="mean").reset_index()


    # Rename columns for clarity
    merged_df_pivot.columns = ["ID", "Avg_PRCP", "Avg_SNWD", "Avg_TMAX", "Avg_TMIN"]


    # Merge station locations back
    merged_df = merged_df_pivot.merge(stations, on="ID", how="left")

    # Load haunted places dataset
    haunted_places = file


    # Extract coordinates from haunted places
    haunted_coords = np.array(haunted_places[["latitude", "longitude"]])
    

    station_coords = np.array(merged_df[["latitude", "longitude"]])

    # Use KDTree for efficient nearest neighbor search
    tree = cKDTree(station_coords)
    distances, nearest_station_indices = tree.query(haunted_coords)

    # Assign nearest station to each haunted place
    haunted_places["nearest_station_id"] = merged_df.iloc[nearest_station_indices]["ID"].values

    # Merge weather data into haunted places dataset
    final_df = haunted_places.merge(merged_df, left_on="nearest_station_id", right_on="ID", suffixes=("_haunted", "_station"))

    # Remove redundant columns
    final_df.drop(columns=["ID", "nearest_station_id", "latitude_station", "longitude_station"], inplace=True, errors="ignore")

    # Compute Diurnal Temperature Range (DTR)
    final_df["Diurnal Temperature Range (DTR)"] = final_df["Avg_TMAX"] - final_df["Avg_TMIN"]

    # Drop TMAX and TMIN as they are no longer needed
    final_df.drop(columns=["Avg_TMAX", "Avg_TMIN"], inplace=True)

    return final_df

    


In [36]:
second_dataset_added = add_weather_dataset(first_dataset_added)



In [37]:
second_dataset_added

,city,country,description,location,state,state_abbrev,longitude_haunted,latitude_haunted,city_longitude,city_latitude,date_occured,geometry,buffer_10_miles,schools_within_10_miles,distance_to_nearest_school_km,is_haunted_place_a_school?,Avg_PRCP,Avg_SNWD,Diurnal Temperature Range (DTR)
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727,2025-02-20,POINT (-9518361.16 5306205.786),"POLYGON ((-9502267.76 5306205.786, -9502345.25...",60,1.326176,No,44.074434,0.388175,117.428082
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434,2025-01-01,POINT (-9393343.839 5156699.978),"POLYGON ((-9377250.439 5156699.978, -9377327.9...",5,3.642689,No,44.074434,0.388175,117.428082
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547,2025-01-01,POINT (-9354806.457 5146690.408),"POLYGON ((-9338713.057 5146690.408, -9338790.5...",18,0.596638,No,44.074434,0.388175,117.428082
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547,1970-02-23,POINT (-9352792.587 5146866.066),"POLYGON ((-9336699.187 5146866.066, -9336776.6...",18,1.104957,No,44.074434,0.388175,117.428082
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097,2025-01-01,POINT (-9433790.006 5197600.789),"POLYGON ((-9417696.606 5197600.789, -9417774.1...",5,0.822609,No,44.074434,0.388175,117.428082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9726,Westminster,United States,at 12 midnight you can see a lady with two lit...,city hall,Colorado,CO,-105.048936,39.862610,-105.037205,39.836653,2025-01-01,POINT (-11693994.064 4845997.164),"POLYGON ((-11677900.664 4845997.164, -11677978...",182,1.216969,No,34.185950,0.000000,128.038889
9727,Westminster,United States,Is haunted by the victims of a murder that hap...,Pillar of Fire,Colorado,CO,-105.032091,39.847237,-105.037205,39.836653,2025-01-01,POINT (-11692118.932 4843767.989),"POLYGON ((-11676025.532 4843767.989, -11676103...",219,0.602155,No,34.185950,0.000000,128.038889
9728,Wheat Ridge,United States,The institution was for kids 18 years old and ...,Ridge Mental Institution,Colorado,CO,-105.063974,39.769726,-105.077206,39.766098,2025-02-18,POINT (-11695668.031 4832535.669),"POLYGON ((-11679574.631 4832535.669, -11679652...",275,1.031467,No,34.185950,0.000000,128.038889
9729,Wheat Ridge,United States,Gymnasium - their have been reports of a litt...,Wheat Ridge Middle School,Colorado,CO,-105.103613,39.764055,-105.077206,39.766098,2025-01-01,POINT (-11700080.635 4831714.386),"POLYGON ((-11683987.235 4831714.386, -11684064...",239,0.031699,Yes,34.185950,0.000000,128.038889
